In [60]:
import requests
import time
import bs4

In [61]:
# Data Accquisition:
links = dict()

with open("data.csv", 'r') as f:
    for line in f:
        lnk = line.strip()
        if not links.get(lnk,False):
            links[lnk] = True

print(f"There are total {len(links)} links")

There are total 19 links


In [96]:
def extract_price(soupPage):
  prc = soupPage.find("div", id="corePriceDisplay_desktop_feature_div")

  if prc:
      prc_div = prc.find("div")  # Locate the inner price container
      if prc_div:
          # Extract main price with currency
          m_prc = prc_div.find("span", class_="a-price aok-align-center reinventPricePriceToPayMargin priceToPay")
          if m_prc:
              price_parts = m_prc.find_all("span", recursive=False)
              currency = price_parts[0].text.strip()
              whole_price = price_parts[1].text.strip()
              main_price = f"{currency}{whole_price}"
          else:
              main_price = None  # Missing main price

          # Extract price per unit details (if available)
          ppu = prc_div.find("span", class_="pricePerUnit")
          if ppu:
              ppu_txt = ppu.text.strip()[1:]  # Remove leading '('
              ppu_parts = ppu_txt.split(" / ")
              if len(ppu_parts) == 2:
                  price_per_unit = ppu_parts[0].strip()
                  unit = ppu_parts[1].strip()[0:-1]  # Remove trailing ')'
              else:
                  price_per_unit = None
                  unit = None
          else:
              price_per_unit = None
              unit = None

          return {
              "main_price": main_price,
              "price_per_unit": price_per_unit,
              "unit": unit
          }
  if prc == None or prc_div == None :
    return {"main_price": "None"}

In [106]:
def extract_details(soupPage):

    # Find the product details table
    product_table = soupPage.find("table", class_="a-normal a-spacing-micro")
    
    # Extract product details
    product_details = {}
    for row in product_table.find_all("tr"):
      labels = row.find_all("td", class_="a-span3")  # Get labels (e.g., Brand, Item weight)
      values = row.find_all("td", class_="a-span9")  # Get corresponding values
    
      # Assuming only one label and value per row
      if labels and values:
        label = labels[0].text.strip()
        value = values[0].text.strip()
        product_details[label] = value
    
    return product_details

In [98]:
def extract_img(soupPage):
    try:
        img_element = soupPage.find("div",id="imgTagWrapperId").find("img")
    except Exception as e:
        pass

    try:
        img_element = soupPage.find("div",id="image-block").find("img")
    except Exception as e:
        print("Could not resolve Prodcut Image:",e)
    
    # Extract image URLs from different sources
    image_urls = []
    
    # 1. URL from 'src' attribute (currently displayed image)
    image_urls.append(img_element["src"])
    
    # 2. URLs from 'data-a-dynamic-image' attribute (various sizes)
    try:
      image_data = eval(img_element["data-a-dynamic-image"])  # Assuming valid JSON format
      for url, _ in image_data.items():
        image_urls.append(url)
    except Exception as e:
        print(e)
    
    return {"Product Images": ", ".join(image_urls)} 

In [107]:
res = [dict() for i in range(len(links))]

In [108]:
ctr = -1
for lnk in links:
    ctr += 1
    
    print("Wating...")
    time.sleep(10)
    print("Executing...")
    
    r = requests.get(lnk)
    if not (200 <= r.status_code < 300):
        continue
    
    soupPage = bs4.BeautifulSoup(r.text,"html.parser")

    res[ctr].update(extract_img(soupPage))
    res[ctr].update(extract_price(soupPage))

    try:
        res[ctr].update(extract_details(soupPage))
    except Exception as e:
        print("Error occured while extracting details:",e)
        
    res[ctr].update({"ASIN" : lnk.split('/')[-1]})
    print(f"Done: {lnk}")

Wating...
Executing...
Could not resolve Prodcut Image: 'NoneType' object has no attribute 'find'
Done: https://www.amazon.com.au/dp/B01BIRVL6A
Wating...
Executing...
Could not resolve Prodcut Image: 'NoneType' object has no attribute 'find'
Done: https://www.amazon.com.au/dp/B00544A9SK
Wating...
Executing...
Could not resolve Prodcut Image: 'NoneType' object has no attribute 'find'
Done: https://www.amazon.com.au/dp/B00NKJZE4K
Wating...
Executing...
Could not resolve Prodcut Image: 'NoneType' object has no attribute 'find'
Done: https://www.amazon.com.au/dp/B08XNS9BFS
Wating...
Executing...
Could not resolve Prodcut Image: 'NoneType' object has no attribute 'find'
Done: https://www.amazon.ca/dp/B0CP9QSBKR
Wating...
Executing...
Could not resolve Prodcut Image: 'NoneType' object has no attribute 'find'
Error occured while extracting details: 'NoneType' object has no attribute 'find_all'
Done: https://www.amazon.co.uk/dp/B0CMCSYHZM
Wating...
Executing...
Wating...
Executing...
Wating...

In [109]:
res

[{'Product Images': 'https://m.media-amazon.com/images/I/A1JANUL33bL.__AC_SX300_SY300_QL70_ML2_.jpg, https://m.media-amazon.com/images/I/A1JANUL33bL._AC_SX466_.jpg, https://m.media-amazon.com/images/I/A1JANUL33bL._AC_SX569_.jpg, https://m.media-amazon.com/images/I/A1JANUL33bL._AC_SY355_.jpg, https://m.media-amazon.com/images/I/A1JANUL33bL._AC_SX425_.jpg, https://m.media-amazon.com/images/I/A1JANUL33bL._AC_SY450_.jpg, https://m.media-amazon.com/images/I/A1JANUL33bL._AC_SX679_.jpg, https://m.media-amazon.com/images/I/A1JANUL33bL._AC_SX522_.jpg',
  'main_price': '$11.00',
  'price_per_unit': '$1.10$1.10',
  'unit': 'count',
  'Brand': 'Lemsip',
  'Item weight': '60 Grams',
  'Item dimensions L x W x H': '50 x 100 x 100 millimetres',
  'Number of items': '1',
  'Item form': 'Powder',
  'ASIN': 'B01BIRVL6A'},
 {'Product Images': 'https://m.media-amazon.com/images/I/81Y03aKsYJL.__AC_SX300_SY300_QL70_ML2_.jpg, https://m.media-amazon.com/images/I/81Y03aKsYJL._AC_SX522_.jpg, https://m.media-ama